<a href="https://colab.research.google.com/github/devsjee/internet-profile-summary/blob/main/Public_Profiles_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, I am going to develop an end to end application that will take a person's name and identification clues and builds a summary about the person from the profiles publicly available about him/her on the Internet.

Rough outline of how I am going to proceed:

*   Figure out the input and its format
*   how to search the internet with this input
*   how to filter out results?
*   there might be false positives how to recognize them? may be have a manual intervention at this point of time.
*   summary creation -- using LLM?
*   false negatives? how much important? how to improvise (this can be handled in the second round of coding too)







**Figure out the input and its format**

The input will definitely contain the name of the person. In some cases like for celebrities (let's say Rajinikanth) the name alone suffices to fetch relevant articles when we do a google search on the internet. But for a normal person (let's say Devi Ganesan ,which is me :) ), the name alone is less likely to fetch only the relevant articles. That is, the search results from google search engine will not only include relevant pages about me but also include pages talking about 'Actor Sivaji Ganesan', 'Goddess Devi', etc. I may not be able to eliminate all the irrelevant articles from the search but at least I would want the relevant articles to come up in the top 5 to 10 results of the search. In technical words, I would like the relevant results to have a high MAP ([Mean Average Precision](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)).

To begin with, I am going to assume that the input will consist of the following:

1.   name of the person
2.   place of birth/residence (optional)
3.   profession/industry name (optional)
4.   educational institute (optional)
5.   any other association that would be of significance (optional)





In [18]:
print("For creating the profile summary, please enter the following details")
name = input("Name of the person (mandatory): ")
place = input("birth/residence place (optional): ")
profession = input("profession/industry name (optional): ")
edu_inst = input("educational institute (optional): ")
other = input("any other significant association (optional): ")



For creating the profile summary, please enter the following details
Name of the person (mandatory): devi ganesan
birth/residence place (optional): chennai
profession/industry name (optional): accenture
educational institute (optional): iitm
any other significant association (optional): wellington


In [20]:
query = " ".join([name,edu_inst, place, profession, other])
print(query)

devi ganesan iitm chennai accenture wellington


In [25]:






#Let's do a web search with the input and gather some results
#!pip install beautifulsoup4
#!pip install google

try:
  from googlesearch import search
except ImportError:
  print("No module named google present. Please install 'google' package using pip")
else:
  print('successfully imported')

selected_webpages=[]

#I am manually selecting those urls from which I want my profile summary to be created -- for experimentation purpose
for webpage in (search(query,tld="com", num=10,lang='en',stop=10, pause=2)):
  sel= input(webpage+" (Y/N) : ")
  if sel == 'y' or sel == 'Y':
    selected_webpages.append(webpage)


successfully imported
https://in.linkedin.com/in/devi-g-310190215 (Y/N) : 
http://www.cse.iitm.ac.in/~gdevi/ (Y/N) : y
https://in.linkedin.com/in/devi-ganesan-52226641 (Y/N) : 
https://www.researchgate.net/profile/Devi_Ganesan (Y/N) : 
https://issuu.com/hindustanuniversitypublications/docs/hcellence_magazine_2022 (Y/N) : 
https://www.inae.in/storage/2022/05/INAE_Year_Book_2022_low.pdf (Y/N) : 
https://www.iitsystem.ac.in/sites/default/files/annualreport/3/IITM-AnnualReport-2015-16.pdf (Y/N) : 
http://ioa-dst.pec.ac.in/all-academicians?page=883 (Y/N) : 
https://ipindia.gov.in/writereaddata/Portal/IPOJournal/1_5007_1/Part-2.pdf (Y/N) : 
https://www.iodglobal.com/masterclass/get_more_data (Y/N) : 


In [26]:

print(selected_webpages)

['http://www.cse.iitm.ac.in/~gdevi/']


So, till this point, I have been able to do a google search for the input query and manually select the relevant urls from a list of search results obtained from google query. Next, I am going to use the WebBasedLoader from langchain framework to fetch the page-contents of the above urls and then use a LLM model (here it is Gemini-pro) again via the langchain framework to create a summary of the fetched pages.


Langchain for text summarization has been already explored [in this colab book](https://colab.research.google.com/gist/devsjee/8bee27534e5a263deb35f05d4ee6b28f/text-summarization-using-langchain-and-gemini-llm.ipynb?authuser=2#scrollTo=zesF2hABSdnV) and the following is a simple adaption of the same.

In [2]:
#uncomment the following lines and run this cell if langchain and langchain_google_genai are not already installed

#!pip install --quiet langchain_google_genai
#!pip install --quiet langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00


In [3]:
import os
import getpass
from langchain import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.prompt_template import format_document
from langchain.schema import StrOutputParser

from langchain.docstore.document import Document

#this google api key can be obtained free of cost from makersuite.com which is Google AI studio
os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter the GOOGLE API KEY: ")

Enter the GOOGLE API KEY: ··········


In [27]:
loader = WebBaseLoader(selected_webpages)
docs = loader.load()
print(docs)


[Document(page_content='\n\n\n\nDevi Ganesan\n\n\n\n\n\n\n\n\n\n\n\nDevi Ganesan\n\n\n\n\n\n\n\n\n\nABOUT ME\nRESEARCH\nPUBLICATIONS\nTALKS\nUSEFUL LINKS\n\n\n\n\n\nContact\n\nDevi Ganesan\r\n\t\t\t\t\tBSB 365,\r\n\t\t\t\t\tArtificial Intelligence and Databases Lab (AIDB),\r\n\t\t\t\t\tDepartment of Computer Science & Engineering,\r\n\t\t\t\t\tIndian Institue of Technology Madras,\r\n\t\t\t\t\tChennai-36, Tamil Nadu, India.\r\n\t\t\t\t\t\r\n\t\t\nEmail: gdevi@cse.iitm.ac.in\n\n\n\n\n\n\nAbout me\n\n One of my favorite quotes is "There are only two ways to live your life - one is as though nothing is a miracle, the other is as though everything is a miracle"- by Albert Einstein. Many a time, I have lost myself in thoughts about the beauty and brilliance of Mother Nature. Hence, I sincerly believe that I belong to the latter category ;) Nevertheless, my motto - "Never Stop Learning" will speak for me.\nPresent\nI am currently engaged in doctoral research at IIT Madras. I work under the s

In [15]:
llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature =0.7, top_p=0.85)
doc_prompt = PromptTemplate.from_template("{page_content}")
llm_prompt = PromptTemplate.from_template("Create a profile summary of the following : {text} \n\n Summary:")

print(doc_prompt)
print(llm_prompt)


input_variables=['page_content'] template='{page_content}'
input_variables=['text'] template='Create a profile summary of the following : {text} \n\n Summary:'


In [28]:

stuff_chain = (
        {"text": lambda docs: "\n\n".join(format_document(doc, doc_prompt) for doc in docs)}
        | llm_prompt | llm | StrOutputParser()
        )

stuff_chain.invoke(docs)

'Devi Ganesan is a PhD researcher at the Indian Institute of Technology Madras, specializing in the intersection of classical and statistical artificial intelligence. Her research interests lie in the confluence of classical and statistical AI. She holds a Master of Technology in CSE from IIT Madras and a Bachelor of Engineering in CSE from Madras Institute of Technology, Anna University. Prior to her doctoral studies, she worked as a Planning & Systems Engineer at Bharat Heavy Electricals Limited -Trichy.'

Some output analysis goes here:

The following are the urls I had selected for summary

['https://in.linkedin.com/in/devi-g-310190215', 'http://www.cse.iitm.ac.in/~gdevi/', 'https://www.researchgate.net/profile/Devi_Ganesan']

And the summary created for me is:

Devi Ganesan is a PhD researcher at the Indian Institute of Technology Madras, specializing in the intersection of classical and statistical artificial intelligence. Her research interests lie in the confluence of classical and statistical AI. She holds a Master of Technology in CSE from IIT Madras and a Bachelor of Engineering in CSE from Madras Institute of Technology, Anna University. Prior to her doctoral studies, she worked as a Planning & Systems Engineer at Bharat Heavy Electricals Limited -Trichy.


I think this is a decent summmary of my profile.

---

There are few things that will have to be investigated to improve the performance of this system.
1. Construction of query from the input variables:
A query such as 'devi ganesan' is very general whereas a query like 'devi ganesan iitm accenture wellington' is too spectific. So, both of these are likely to fetch many irrelevant results. One idea is to construct several inputs like 'devi ganesan iitm', 'devi ganesan accenture', 'devi ganesan wellington' to perform google search and then select the most useful results.

2. From manual to automated filtering: Currently, I am filtering out the useful relevant links from the listed ones. One idea for automation is to select the common urls among the multiple searches explained in the previous point.

3. Suppose the input is very sparse, can the LLM indulge in hallucinations? This is to be verified and if found true, one solution would be to use a vector database for narrowing down the summarization scope to the stored articles of the vector database only.